## GENERATE LABELS FOR TRAINING DATA

In [ ]:
import csv
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv('../train_data/bizfeatures_meanfeature.csv')
label_df = pd.read_csv('train.csv')

In [ ]:
df = df.sort_values(by='business_id')
df

In [ ]:
df = df.sort_values(by='business_id')
label_df = label_df.sort_values(by='business_id')

In [ ]:
#randomly breaking the dataset into 80:20 ratio for training and testing
msk = np.random.rand(len(df)) < 0.8
train_new_model = df[msk]
test_new_model = df[~msk]
label_train_new_model = label_df[msk]
label_test_new_model = label_df[~msk]

In [ ]:
def str_to_nplist(string):
    string = string.replace("[", "")
    string = string.replace("]", "")
    string = string.split(',')
    return np.asarray(string).astype(np.float32)

In [ ]:
mean_features = np.asarray([str_to_nplist(i) for i in train_new_model['feature']])
labels = label_train_new_model['labels'].values

In [ ]:
mean_features_test = np.asarray([str_to_nplist(i) for i in test_new_model['feature']])
labels_test = label_test_new_model['labels'].values

In [ ]:
X = mean_features
Y = labels

In [ ]:
label_binarizer = MultiLabelBinarizer()

In [ ]:
def convertlabelsfromString(s, ind):
    if type(s) == str:
        s = s.split(" ")
        s = [int(x) for x in s]
        return s
    else:
        return []

In [ ]:
Y = [convertlabelsfromString(Y[i],i) for i in range(Y.shape[0])]

In [ ]:
Y = label_binarizer.fit_transform(Y)

In [ ]:
X.shape

In [ ]:
classifier = OneVsRestClassifier(svm.SVC(kernel='linear'))
classifier.fit(X, Y)

In [ ]:
prediction = classifier.predict(mean_features_test)
values = label_binarizer.inverse_transform(prediction)

In [ ]:
inv_values = label_binarizer.fit_transform(values)

In [ ]:
Y1 = [convertlabelsfromString(labels_test[i],i) for i in range(labels_test.shape[0])]

In [ ]:
Y1 = label_binarizer.fit_transform(Y1)

In [ ]:
f1_score(Y1,inv_values,average='macro')

## GENERATE LABELS FOR TEST DATA

In [ ]:
df = pd.read_csv('../train_data/bizfeatures_meanfeature_test.csv')
df = df.sort_values(by='business_id')

In [ ]:
mean_features_test = np.asarray([str_to_nplist(i) for i in df['feature']])
X = mean_features_test

In [ ]:
prediction = classifier.predict(X)
values = label_binarizer.inverse_transform(prediction)

In [ ]:
values